In [ ]:
! pip install pylatexenc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 8.5 MB/s 
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136835 sha256=cebee716c4f8e3f9ce635c86af82ff4aacf6987b88b273cc9b8425f993fe7c6e
  Stored in directory: /root/.cache/pip/wheels/f1/8a/f5/33ee79d4473eb201b519fa40f989b842e373237395a3421f52
Successfully built pylatexenc


In [ ]:
! pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/qiskit-community/qiskit-textbook.git to /tmp/pip-req-build-vo4anmjd
  Running command git clone -q https://github.com/qiskit-community/qiskit-textbook.git /tmp/pip-req-build-vo4anmjd
     |████████████████████████████████| 6.5 MB 6.1 MB/s 
     |████████████████████████████████| 18.0 MB 321 kB/s 
     |████████████████████████████████| 240 kB 59.1 MB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 112 kB 54.5 MB/s 
     |████████████████████████████████| 1.6 MB 60.4 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 943 kB 74.9 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 


In [ ]:
from qiskit.providers.aer import AerSimulator
from qiskit import *

In [ ]:
from qiskit import IBMQ

In [ ]:
IBMQ.save_account('5c68361d5b9e2c398434de7d981228d3556831658cded8c26a033c8ac53da6315d5323659c57a779fe4630f4fd9d3104ad65483f905c39767a8c8d30caa5a94f')

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_oslo

In [ ]:
qcomp = provider.get_backend('simulator_stabilizer')

In [ ]:
from qiskit.tools.monitor import job_monitor

## Utility

#### ARX rounds

##### Addition utility function

In [ ]:
def add(n):
  a = QuantumRegister(n,'a')
  b = QuantumRegister(n,'b')
  ancilla = QuantumRegister(1, 'ancilla')
  z = QuantumRegister(1, 'z')
  qc = QuantumCircuit (a, b, ancilla, z, name="add")
  # qc = QuantumCircuit (a, b, ancilla, name="add")
  # qc.draw(output='mpl')

  for i in range(1,n):
    qc.cx(a[i],b[i])

  qc.cx(a[1], ancilla)
  qc.ccx(a[0], b[0], ancilla)
  qc.cx(a[2], a[1])
  qc.ccx(ancilla, b[1], a[1])
  qc.cx(a[3], a[2])

  for i in range(2,n-2):        
    qc.ccx(a[i-1],b[i],a[i])
    qc.cx(a[i+2],a[i+1])

  qc.ccx(a[n-3],b[n-2],a[n-2])
  qc.cx(a[n-1],z)

  qc.ccx(a[n-2],b[n-1],z)
  for i in range(1, n-1):
    qc.x(b[i])

  qc.cx(ancilla, b[1])
  for i in range(2, n):
    qc.cx(a[i-1],b[i])

  qc.ccx(a[n-3],b[n-2],a[n-2])
  for i in range(n-3,1,-1):
    qc.ccx(a[i-1],b[i],a[i])
    qc.cx(a[i+2],a[i+1])
    qc.x(b[i+1])

  qc.ccx(ancilla,b[1],a[1])
  qc.cx(a[3],a[2])
  qc.x(b[2])

  qc.ccx(a[0],b[0],ancilla)
  qc.cx(a[2],a[1])
  qc.x(b[1])

  qc.cx(a[1],ancilla)

  for i in range(0,n):
    qc.cx(a[i],b[i])

  # qc.measure_all()
  # qc.draw(output='mpl', filename="add_"+str(n))
  return qc

In [ ]:
def additionMod_2n(n):           # a,b..........a,b=a+b
  a = QuantumRegister(n,'a')
  b = QuantumRegister(n,'b')
  ancilla = QuantumRegister(1, 'ancilla')
  qc = QuantumCircuit (a, b, ancilla,  name="add_mod_32")

  l=list(range(n*2+1))
  l.remove(n-1)
  l.remove(n*2-1)

  add_circuit=add(n-1).to_gate()
  # print("additionMod32")
  qc.append(add_circuit, l+[n*2-1])
  qc.cx(a[n-1],b[n-1])

  return qc

##### XOR utility function

In [ ]:
def XOR(n):             # a,b ............ a=a^b,b
  a=QuantumRegister(n,'a')
  b=QuantumRegister(n,'b')
  qc=QuantumCircuit(a,b, name="XOR")
  for i in range(n):
    qc.cx(b[i],a[i])

  # qc.draw(output='mpl', filename='XOR')
  
  return qc

##### Rotation utility

In [ ]:
def classical_n_rotateBy_d(arr,n,d):
  arr = arr[d:]+arr[:d]
  return arr

#### one arx

In [ ]:
def one_arx(n, shift, d_l):
  w=QuantumRegister(n, 'w')
  x=QuantumRegister(n, 'x')
  y=QuantumRegister(n, 'y')
  z=QuantumRegister(n, 'z')
  ancilla = QuantumRegister(1, 'ancilla')

  qc=QuantumCircuit(w,x,y,z,ancilla,name="one arx")

  w_l = list(range(n*0,n*1))
  x_l = list(range(n*1,n*2))
  y_l = list(range(n*2,n*3))
  z_l = list(range(n*3,n*4))
  ancilla_l = [n*4]
  
  # print("w+=x: x_l",x_l,"w_l",w_l)
  qc.append(additionMod_2n(n).to_gate(), x_l+w_l+ancilla_l)        # a+=b   ---> w+=x
  # print("z^=w: z_l",z_l,"w_l",w_l)
  qc.append(XOR(n).to_gate(), z_l+w_l)                             # d^=a   ---> z^=w
  # qc.append(n_rotateBy_d(n, shift).to_gate(), z_l)                 # d<<<16 ---> z

  # print("d<<shift: d_l",d_l)
  # d_l = classical_n_rotateBy_d(d_l,n,shift)
  # print("After: d_l",d_l)


  return qc

#### initialize

In [ ]:
def init(row0, row1, row2, row3, base, n):
  A0,A1,A2,A3 = row0.split()
  B0,B1,B2,B3 = row1.split()
  C0,C1,C2,C3 = row2.split()
  D0,D1,D2,D3 = row3.split()

  A0=bin(int(A0, base))[2:].zfill(n)            # 16 for hexadecimal value
  B0=bin(int(B0, base))[2:].zfill(n)
  C0=bin(int(C0, base))[2:].zfill(n)
  D0=bin(int(D0, base))[2:].zfill(n)

  A1=bin(int(A1, base))[2:].zfill(n)
  B1=bin(int(B1, base))[2:].zfill(n)
  C1=bin(int(C1, base))[2:].zfill(n)
  D1=bin(int(D1, base))[2:].zfill(n)

  A2=bin(int(A2, base))[2:].zfill(n)
  B2=bin(int(B2, base))[2:].zfill(n)
  C2=bin(int(C2, base))[2:].zfill(n)
  D2=bin(int(D2, base))[2:].zfill(n)

  A3=bin(int(A3, base))[2:].zfill(n)
  B3=bin(int(B3, base))[2:].zfill(n)
  C3=bin(int(C3, base))[2:].zfill(n)
  D3=bin(int(D3, base))[2:].zfill(n)

  return A0,A1,A2,A3,B0,B1,B2,B3,C0,C1,C2,C3,D0,D1,D2,D3

#### measure

In [ ]:
def get_num(ori_num, my_list):
  num=""
  for i in my_list:
    num+=ori_num[i]
  num

  return num[::-1]

In [ ]:
# from qiskit.quantum_info import Statevector

In [ ]:
def measure(qc, n_shots=1, Hex=True, Binary=True, probability=False):#, i=0,j=0,k=0,l=0,A=0,B=0,C=0,D=0,ancilla=0):
  # Select the AerSimulator from the Aer provider
  simulator = AerSimulator(method='matrix_product_state')  
  # simulator = AerSimulator(method='statevector')

  # Run and get counts, using the matrix_product_state method
  tcirc = transpile(qc, simulator)
  result = simulator.run(tcirc, shots=n_shots).result()
  counts = result.get_counts(0)
  print(counts)
  for key in counts.keys():
      print("Count:",counts[key])
      x=key[-4::-1]
      
      print('index_X: ', get_num(x,index_X_l))

      print('X')
      if Hex:
        print(hex(int(get_num(x,a0_l), 2)), hex(int(get_num(x,a1_l), 2)), hex(int(get_num(x,a2_l), 2)), hex(int(get_num(x,a3_l), 2)))
        print(hex(int(get_num(x,b0_l), 2)), hex(int(get_num(x,b1_l), 2)), hex(int(get_num(x,b2_l), 2)), hex(int(get_num(x,b3_l), 2)))
        print(hex(int(get_num(x,c0_l), 2)), hex(int(get_num(x,c1_l), 2)), hex(int(get_num(x,c2_l), 2)), hex(int(get_num(x,c3_l), 2)))
        print(hex(int(get_num(x,d0_l), 2)), hex(int(get_num(x,d1_l), 2)), hex(int(get_num(x,d2_l), 2)), hex(int(get_num(x,d3_l), 2)))
        print()
      if Binary:
        print(get_num(x,a0_l), get_num(x,a1_l), get_num(x,a2_l), get_num(x,a3_l))
        print(get_num(x,b0_l), get_num(x,b1_l), get_num(x,b2_l), get_num(x,b3_l))
        print(get_num(x,c0_l), get_num(x,c1_l), get_num(x,c2_l), get_num(x,c3_l))
        print(get_num(x,d0_l), get_num(x,d1_l), get_num(x,d2_l), get_num(x,d3_l))
        print()

      print()


In [ ]:
def measure_2(qc, n_shots=1, Hex=True, Binary=True, probability=False):#, i=0,j=0,k=0,l=0,A=0,B=0,C=0,D=0,ancilla=0):
  # Select the AerSimulator from the Aer provider
  simulator = AerSimulator(method='matrix_product_state')  
  # simulator = AerSimulator(method='statevector')

  # Run and get counts, using the matrix_product_state method
  tcirc = transpile(qc, simulator)
  result = simulator.run(tcirc, shots=n_shots).result()
  counts = result.get_counts(0)
  print(counts)

In [ ]:
# def measure_3(qc, n_shots=1, Hex=True, Binary=True, probability=False,backend=None):#, i=0,j=0,k=0,l=0,A=0,B=0,C=0,D=0,ancilla=0):
#   job=execute(qc,backend=backend)

#   # Run and get counts, using the matrix_product_state method
#   tcirc = transpile(qc, simulator)
#   result = simulator.run(tcirc, shots=n_shots).result()
#   counts = result.get_counts(0)
#   print(counts)

In [ ]:
def measure_counts(qc, n, n_shots=1):#, i=0,j=0,k=0,l=0,A=0,B=0,C=0,D=0,ancilla=0):
  # Select the AerSimulator from the Aer provider
  simulator = AerSimulator(method='matrix_product_state')  
  # simulator = AerSimulator(method='statevector')

  # Run and get counts, using the matrix_product_state method
  tcirc = transpile(qc, simulator)
  result = simulator.run(tcirc, shots=n_shots).result()
  counts = result.get_counts(0)
  # print(counts)
  return counts

#### rotate RIGHT

In [ ]:
def classical_n_rotateBy_d_RIGHT(arr,n,d):
  arr = arr[-d:]+arr[:-d]
  return arr

# Input

#### Take inputs

In [ ]:
n=8
shift0 = 4
shift1 = 3
shift2 = 2
shift3 = 1
row0 = "01100101 01111000 01110000 01100001 "
row1 = "01110101 01000010 00001000 11100111"
row2 = "11111111 01011111 11100011 10101111"
row3 = "00010101 00111100 10101111 10111111"
base = 2
n_rounds = 4

In [ ]:
# n=32
# shift0 = 16
# shift1 = 8
# shift2 = 12
# shift3 = 7
# row0 = "61707865 3320646e 79622d32 6b206574"
# row1 = "03020100 07060504 0b0a0908 0f0e0d0c"
# row2 = "13121110 17161514 1b1a1918 1f1e1d1c"
# row3 = "00000001 00000000 4a000000 00000000"

In [ ]:
# n=5
# shift0 = 4
# shift1 = 2
# shift2 = 3
# shift3 = 1
# row0 = "01011 00111 01110 10100"
# row1 = "00101 01010 00001 11100"
# row2 = "11111 01011 11100 10101"
# row3 = "00101 00100 10101 10111"
# base = 2

# Getting PNB

##Circuit

#### Circuit

In [ ]:
# registers for X
a0=QuantumRegister(n, 'a0')
a1=QuantumRegister(n, 'a1')
a2=QuantumRegister(n, 'a2')
a3=QuantumRegister(n, 'a3')

b0=QuantumRegister(n, 'b0')
b1=QuantumRegister(n, 'b1')
b2=QuantumRegister(n, 'b2')
b3=QuantumRegister(n, 'b3')

c0=QuantumRegister(n, 'c0')
c1=QuantumRegister(n, 'c1')
c2=QuantumRegister(n, 'c2')
c3=QuantumRegister(n, 'c3')

d0=QuantumRegister(n, 'd0')
d1=QuantumRegister(n, 'd1')
d2=QuantumRegister(n, 'd2')
d3=QuantumRegister(n, 'd3')

ancilla0 = QuantumRegister(1, 'ancilla0')
ancilla1 = QuantumRegister(1, 'ancilla1')
ancilla2 = QuantumRegister(1, 'ancilla2')
ancilla3 = QuantumRegister(1, 'ancilla3')

index_X = QuantumRegister(1, "index_X")

cR = ClassicalRegister(1, 'cR')

qc=QuantumCircuit(a0,a1,a2,a3,b0,b1,b2,b3,c0,c1,c2,c3,d0,d1,d2,d3,ancilla0,ancilla1,ancilla2,ancilla3,index_X,cR)

#### initialising lists

In [ ]:
a0_l=list(range(n*0,n*1))        # A0 block
b0_l=list(range(n*4,n*5))        # B0 block
c0_l=list(range(n*8,n*9))        # C0 block
d0_l=list(range(n*12,n*13))        # D0 block
ancilla0_l=list([n*16+0])

a1_l=list(range(n*1,n*2))        # A1 block
b1_l=list(range(n*5,n*6))        # B1 block
c1_l=list(range(n*9,n*10))        # C1 block
d1_l=list(range(n*13,n*14))        # D1 block
ancilla1_l=list([n*16+1])

a2_l=list(range(n*2,n*3))        # A2 block
b2_l=list(range(n*6,n*7))        # B2 block
c2_l=list(range(n*10,n*11))        # C2 block
d2_l=list(range(n*14,n*15))        # D2 block
ancilla2_l=list([n*16+2])

a3_l=list(range(n*3,n*4))        # A3 block
b3_l=list(range(n*7,n*8))        # B3 block
c3_l=list(range(n*11,n*12))        # C3 block
d3_l=list(range(n*15,n*16))        # D3 block
ancilla3_l=list([n*16+3])


temp=n*16+3+1


In [ ]:
index_X_l=[temp]

#### initialise states

In [ ]:
A0,A1,A2,A3,B0,B1,B2,B3,C0,C1,C2,C3,D0,D1,D2,D3 = init(row0, row1, row2, row3, base, n)

###### column 0

In [ ]:
A0=A0[::-1]
B0=B0[::-1]
C0=C0[::-1]
D0=D0[::-1]

for i in range(len(A0)):
  if A0[i]=='1':
    qc.x(a0[i])

for i in range(len(B0)):        
  if B0[i]=='1':
    qc.x(b0[i])

for i in range(len(C0)):
  if C0[i]=='1':
    qc.x(c0[i])

for i in range(len(D0)):
  if D0[i]=='1':
    qc.x(d0[i])


###### column 1

In [ ]:
A1=A1[::-1]
B1=B1[::-1]
C1=C1[::-1]
D1=D1[::-1]

for i in range(len(A1)):
  if A1[i]=='1':
    qc.x(a1[i])

for i in range(len(B1)):
  if B1[i]=='1':
    qc.x(b1[i])

for i in range(len(C1)):
  if C1[i]=='1':
    qc.x(c1[i])

for i in range(len(D1)):
  if D1[i]=='1':
    qc.x(d1[i])


###### column 2

In [ ]:
A2=A2[::-1]
B2=B2[::-1]
C2=C2[::-1]
D2=D2[::-1]

for i in range(len(A2)):
  if A2[i]=='1':
    qc.x(a2[i])

for i in range(len(B2)):
  if B2[i]=='1':
    qc.x(b2[i])

for i in range(len(C2)):
  if C2[i]=='1':
    qc.x(c2[i])

for i in range(len(D2)):
  if D2[i]=='1':
    qc.x(d2[i])


###### column 3

In [ ]:
A3=A3[::-1]
B3=B3[::-1]
C3=C3[::-1]
D3=D3[::-1]

for i in range(len(A3)):
  if A3[i]=='1':
    qc.x(a3[i])

for i in range(len(B3)):
  if B3[i]=='1':
    qc.x(b3[i])

for i in range(len(C3)):
  if C3[i]=='1':
    qc.x(c3[i])

for i in range(len(D3)):
  if D3[i]=='1':
    qc.x(d3[i])

#### hadamard on some bits

In [ ]:
for i in range(8):
  qc.h(a3[i])

#### hadamard on index_X; applying ID based on index_X  
ID = 1st counter bit

In [ ]:
qc.h(index_X)
qc.cx(index_X, d0[0])

#### applying arx for r forward rounds

In [ ]:
forward_rounds= 4   # how many forward rounds
total_rounds= n_rounds

In [ ]:

for i in range(0,forward_rounds,2):
  print(i, 'column round')
  # print('column 0')
  qc.append(one_arx(n,shift0, d0_l), a0_l+b0_l+c0_l+d0_l+ancilla0_l)           # quarter round on column 0
  d0_l = classical_n_rotateBy_d(d0_l,n,shift0)

  qc.append(one_arx(n,shift1, b0_l), c0_l+d0_l+a0_l+b0_l+ancilla0_l)
  b0_l = classical_n_rotateBy_d(b0_l,n,shift1)

  qc.append(one_arx(n,shift2, d0_l), a0_l+b0_l+c0_l+d0_l+ancilla0_l)
  d0_l = classical_n_rotateBy_d(d0_l,n,shift2)

  qc.append(one_arx(n,shift3, b0_l), c0_l+d0_l+a0_l+b0_l+ancilla0_l)
  b0_l = classical_n_rotateBy_d(b0_l,n,shift3)

  # print('column 1')
  qc.append(one_arx(n,shift0, d1_l), a1_l+b1_l+c1_l+d1_l+ancilla1_l)            # quarter round on column 1
  d1_l = classical_n_rotateBy_d(d1_l,n,shift0)
  qc.append(one_arx(n,shift1, b1_l), c1_l+d1_l+a1_l+b1_l+ancilla1_l)
  b1_l = classical_n_rotateBy_d(b1_l,n,shift1)
  qc.append(one_arx(n,shift2, d1_l), a1_l+b1_l+c1_l+d1_l+ancilla1_l)
  d1_l = classical_n_rotateBy_d(d1_l,n,shift2)
  qc.append(one_arx(n,shift3, b1_l), c1_l+d1_l+a1_l+b1_l+ancilla1_l)
  b1_l = classical_n_rotateBy_d(b1_l,n,shift3)


  # print('column 2')
  qc.append(one_arx(n,shift0, d2_l), a2_l+b2_l+c2_l+d2_l+ancilla2_l)            # quarter round on column 2
  d2_l = classical_n_rotateBy_d(d2_l,n,shift0)
  qc.append(one_arx(n,shift1, b2_l), c2_l+d2_l+a2_l+b2_l+ancilla2_l)
  b2_l = classical_n_rotateBy_d(b2_l,n,shift1)
  qc.append(one_arx(n,shift2, d2_l), a2_l+b2_l+c2_l+d2_l+ancilla2_l)
  d2_l = classical_n_rotateBy_d(d2_l,n,shift2)
  qc.append(one_arx(n,shift3, b2_l), c2_l+d2_l+a2_l+b2_l+ancilla2_l)
  b2_l = classical_n_rotateBy_d(b2_l,n,shift3)

  # print('column 3')
  qc.append(one_arx(n,shift0, d3_l), a3_l+b3_l+c3_l+d3_l+ancilla3_l)            # quarter round on column 3
  d3_l = classical_n_rotateBy_d(d3_l,n,shift0)
  qc.append(one_arx(n,shift1, b3_l), c3_l+d3_l+a3_l+b3_l+ancilla3_l)
  b3_l = classical_n_rotateBy_d(b3_l,n,shift1)
  qc.append(one_arx(n,shift2, d3_l), a3_l+b3_l+c3_l+d3_l+ancilla3_l)
  d3_l = classical_n_rotateBy_d(d3_l,n,shift2)
  qc.append(one_arx(n,shift3, b3_l), c3_l+d3_l+a3_l+b3_l+ancilla3_l)
  b3_l = classical_n_rotateBy_d(b3_l,n,shift3)

  print(i, 'diagonal round')
  # print('diagonal 0')
  qc.append(one_arx(n,shift0, d3_l), a0_l+b1_l+c2_l+d3_l+ancilla0_l)            # a0,b1,c2,d3    #quarter round on diagonal 0
  d3_l = classical_n_rotateBy_d(d3_l,n,shift0)
  qc.append(one_arx(n,shift1, b1_l), c2_l+d3_l+a0_l+b1_l+ancilla0_l)  
  b1_l = classical_n_rotateBy_d(b1_l,n,shift1)          
  qc.append(one_arx(n,shift2, d3_l), a0_l+b1_l+c2_l+d3_l+ancilla0_l)  
  d3_l = classical_n_rotateBy_d(d3_l,n,shift2)         
  qc.append(one_arx(n,shift3, b1_l), c2_l+d3_l+a0_l+b1_l+ancilla0_l)  
  b1_l = classical_n_rotateBy_d(b1_l,n,shift3)      

  # print('diagonal 1')      
  qc.append(one_arx(n,shift0, d0_l), a1_l+b2_l+c3_l+d0_l+ancilla1_l)            # a1,b2,c3,d0    #quarter round on diagonal 1
  d0_l = classical_n_rotateBy_d(d0_l,n,shift0)
  qc.append(one_arx(n,shift1, b2_l), c3_l+d0_l+a1_l+b2_l+ancilla1_l) 
  b2_l = classical_n_rotateBy_d(b2_l,n,shift1)
  qc.append(one_arx(n,shift2, d0_l), a1_l+b2_l+c3_l+d0_l+ancilla1_l)    
  d0_l = classical_n_rotateBy_d(d0_l,n,shift2)
  qc.append(one_arx(n,shift3, b2_l), c3_l+d0_l+a1_l+b2_l+ancilla1_l)  
  b2_l = classical_n_rotateBy_d(b2_l,n,shift3)

  # print('diagonal 2')
  qc.append(one_arx(n,shift0, d1_l), a2_l+b3_l+c0_l+d1_l+ancilla2_l)            # a2,b3,c0,d1    #quarter round on diagonal 2
  d1_l = classical_n_rotateBy_d(d1_l,n,shift0)
  qc.append(one_arx(n,shift1, b3_l), c0_l+d1_l+a2_l+b3_l+ancilla2_l)  
  b3_l = classical_n_rotateBy_d(b3_l,n,shift1)
  qc.append(one_arx(n,shift2, d1_l), a2_l+b3_l+c0_l+d1_l+ancilla2_l)   
  d1_l = classical_n_rotateBy_d(d1_l,n,shift2)
  qc.append(one_arx(n,shift3, b3_l), c0_l+d1_l+a2_l+b3_l+ancilla2_l)    
  b3_l = classical_n_rotateBy_d(b3_l,n,shift3)

  # print('diagonal 3')
  qc.append(one_arx(n,shift0, d2_l), a3_l+b0_l+c1_l+d2_l+ancilla3_l)            # a3,b0,c1,d2   #quarter round on diagonal 3
  d2_l = classical_n_rotateBy_d(d2_l,n,shift0)
  qc.append(one_arx(n,shift1, b0_l), c1_l+d2_l+a3_l+b0_l+ancilla3_l)    
  b0_l = classical_n_rotateBy_d(b0_l,n,shift1)
  qc.append(one_arx(n,shift2, d2_l), a3_l+b0_l+c1_l+d2_l+ancilla3_l)     
  d2_l = classical_n_rotateBy_d(d2_l,n,shift2)
  qc.append(one_arx(n,shift3, b0_l), c1_l+d2_l+a3_l+b0_l+ancilla3_l) 
  b0_l = classical_n_rotateBy_d(b0_l,n,shift3)


0 column round
0 diagonal round
2 column round
2 diagonal round


#### z gate on the OD bit

row = 2, col=0, bit=0

In [ ]:
# qc.z(c0[0])
qc.z(c0[1])

#### reverse circuit

Reverse total rounds- forward rounds

In [ ]:
for i in range(0, forward_rounds, 2):
  print(i, 'diagonal round')

  # print('diagonal 3')

  b0_l = classical_n_rotateBy_d_RIGHT(b0_l,n,shift3)
  qc.append(one_arx(n,shift3, b0_l).inverse(),c1_l+d2_l+a3_l+b0_l+ancilla3_l)  
  d2_l = classical_n_rotateBy_d_RIGHT(d2_l,n,shift2) 
  qc.append(one_arx(n,shift2, d2_l).inverse(),a3_l+b0_l+c1_l+d2_l+ancilla3_l)     
  b0_l = classical_n_rotateBy_d_RIGHT(b0_l,n,shift1)  
  qc.append(one_arx(n,shift1, b0_l).inverse(),c1_l+d2_l+a3_l+b0_l+ancilla3_l)
  d2_l = classical_n_rotateBy_d_RIGHT(d2_l,n,shift0)
  qc.append(one_arx(n,shift0, d2_l).inverse(),a3_l+b0_l+c1_l+d2_l+ancilla3_l)            # a3,b0,c1,d2   #quarter round on diagonal 3

  # print('diagonal 2')
    
  b3_l = classical_n_rotateBy_d_RIGHT(b3_l,n,shift3)
  qc.append(one_arx(n,shift3, b3_l).inverse(),c0_l+d1_l+a2_l+b3_l+ancilla2_l)  
  d1_l = classical_n_rotateBy_d_RIGHT(d1_l,n,shift2)
  qc.append(one_arx(n,shift2, d1_l).inverse(),a2_l+b3_l+c0_l+d1_l+ancilla2_l) 
  b3_l = classical_n_rotateBy_d_RIGHT(b3_l,n,shift1)
  qc.append(one_arx(n,shift1, b3_l).inverse(),c0_l+d1_l+a2_l+b3_l+ancilla2_l)    
  d1_l = classical_n_rotateBy_d_RIGHT(d1_l,n,shift0)
  qc.append(one_arx(n,shift0, d1_l).inverse(),a2_l+b3_l+c0_l+d1_l+ancilla2_l)            # a2,b3,c0,d1    #quarter round on diagonal 2


  # print('diagonal 1')      
  b2_l = classical_n_rotateBy_d_RIGHT(b2_l,n,shift3)  
  qc.append(one_arx(n,shift3, b2_l).inverse(),c3_l+d0_l+a1_l+b2_l+ancilla1_l)  
  d0_l = classical_n_rotateBy_d_RIGHT(d0_l,n,shift2)
  qc.append(one_arx(n,shift2, d0_l).inverse(),a1_l+b2_l+c3_l+d0_l+ancilla1_l) 
  b2_l = classical_n_rotateBy_d_RIGHT(b2_l,n,shift1)
  qc.append(one_arx(n,shift1, b2_l).inverse(),c3_l+d0_l+a1_l+b2_l+ancilla1_l)  
  d0_l = classical_n_rotateBy_d_RIGHT(d0_l,n,shift0)
  qc.append(one_arx(n,shift0, d0_l).inverse(),a1_l+b2_l+c3_l+d0_l+ancilla1_l)            # a1,b2,c3,d0    #quarter round on diagonal 1

  # print('diagonal 0')

  b1_l = classical_n_rotateBy_d_RIGHT(b1_l,n,shift3)          
  qc.append(one_arx(n,shift3, b1_l).inverse(),c2_l+d3_l+a0_l+b1_l+ancilla0_l)    
  d3_l = classical_n_rotateBy_d_RIGHT(d3_l,n,shift2)       
  qc.append(one_arx(n,shift2, d3_l).inverse(),a0_l+b1_l+c2_l+d3_l+ancilla0_l)   
  b1_l = classical_n_rotateBy_d_RIGHT(b1_l,n,shift1)   
  qc.append(one_arx(n,shift1, b1_l).inverse(),c2_l+d3_l+a0_l+b1_l+ancilla0_l)  
  d3_l = classical_n_rotateBy_d_RIGHT(d3_l,n,shift0)
  qc.append(one_arx(n,shift0, d3_l).inverse(),a0_l+b1_l+c2_l+d3_l+ancilla0_l)            # a0,b1,c2,d3    #quarter round on diagonal 0  


  print(i, 'column round')


  # print('column 3')

  b3_l = classical_n_rotateBy_d_RIGHT(b3_l,n,shift3)
  qc.append(one_arx(n,shift3, b3_l).inverse(),c3_l+d3_l+a3_l+b3_l+ancilla3_l)
  d3_l = classical_n_rotateBy_d_RIGHT(d3_l,n,shift2)
  qc.append(one_arx(n,shift2, d3_l).inverse(),a3_l+b3_l+c3_l+d3_l+ancilla3_l)
  b3_l = classical_n_rotateBy_d_RIGHT(b3_l,n,shift1)
  qc.append(one_arx(n,shift1, b3_l).inverse(),c3_l+d3_l+a3_l+b3_l+ancilla3_l)
  d3_l = classical_n_rotateBy_d_RIGHT(d3_l,n,shift0)
  qc.append(one_arx(n,shift0, d3_l).inverse(),a3_l+b3_l+c3_l+d3_l+ancilla3_l)            # quarter round on column 3

  # print('column 2')
  b2_l = classical_n_rotateBy_d_RIGHT(b2_l,n,shift3)
  qc.append(one_arx(n,shift3, b2_l).inverse(),c2_l+d2_l+a2_l+b2_l+ancilla2_l)
  d2_l = classical_n_rotateBy_d_RIGHT(d2_l,n,shift2)
  qc.append(one_arx(n,shift2, d2_l).inverse(),a2_l+b2_l+c2_l+d2_l+ancilla2_l)
  b2_l = classical_n_rotateBy_d_RIGHT(b2_l,n,shift1)
  qc.append(one_arx(n,shift1, b2_l).inverse(),c2_l+d2_l+a2_l+b2_l+ancilla2_l)
  d2_l = classical_n_rotateBy_d_RIGHT(d2_l,n,shift0)
  qc.append(one_arx(n,shift0, d2_l).inverse(),a2_l+b2_l+c2_l+d2_l+ancilla2_l)            # quarter round on column 2

  # print('column 1')

  b1_l = classical_n_rotateBy_d_RIGHT(b1_l,n,shift3)
  qc.append(one_arx(n,shift3, b1_l).inverse(),c1_l+d1_l+a1_l+b1_l+ancilla1_l)
  d1_l = classical_n_rotateBy_d_RIGHT(d1_l,n,shift2)
  qc.append(one_arx(n,shift2, d1_l).inverse(),a1_l+b1_l+c1_l+d1_l+ancilla1_l)
  b1_l = classical_n_rotateBy_d_RIGHT(b1_l,n,shift1)
  qc.append(one_arx(n,shift1, b1_l).inverse(),c1_l+d1_l+a1_l+b1_l+ancilla1_l)
  d1_l = classical_n_rotateBy_d_RIGHT(d1_l,n,shift0)
  qc.append(one_arx(n,shift0, d1_l).inverse(),a1_l+b1_l+c1_l+d1_l+ancilla1_l)            # quarter round on column 1

  # print('column 0')
  b0_l = classical_n_rotateBy_d_RIGHT(b0_l,n,shift3)
  qc.append(one_arx(n,shift3, b0_l).inverse(),c0_l+d0_l+a0_l+b0_l+ancilla0_l)
  d0_l = classical_n_rotateBy_d_RIGHT(d0_l,n,shift2)
  qc.append(one_arx(n,shift2, d0_l).inverse(),a0_l+b0_l+c0_l+d0_l+ancilla0_l)
  b0_l = classical_n_rotateBy_d_RIGHT(b0_l,n,shift1)
  qc.append(one_arx(n,shift1, b0_l).inverse(),c0_l+d0_l+a0_l+b0_l+ancilla0_l)
  d0_l = classical_n_rotateBy_d_RIGHT(d0_l,n,shift0)
  qc.append(one_arx(n,shift0, d0_l).inverse(),a0_l+b0_l+c0_l+d0_l+ancilla0_l)           # quarter round on column 0
         

0 diagonal round
0 column round
2 diagonal round
2 column round


#### inverting the id bit

In [ ]:
qc.cx(index_X,d0[0])

#### hadamard gate on the index_X

In [ ]:
qc.h(index_X)

#### measure

In [ ]:
qc2=qc.copy()
qc2.measure(index_X,cR[0])

In [ ]:
qcomp=provider.get_backend('simulator_mps')
job=execute(qc2,backend=qcomp)
job_monitor(job)
result=job.result()
result.get_counts(qc2)

Job Status: job incurred error     


JSONDecodeError: ignored

In [ ]:
qcomp=provider.get_backend('simulator_mps')

In [ ]:
job=execute(qc2,backend=qcomp)

In [ ]:
job_monitor(job)

Job Status: job has successfully run


In [ ]:
result=job.result()

In [ ]:
result.get_counts(qc2)

{'0': 975, '1': 3025}

In [ ]:
qc2=qc.copy()
qc2.measure(index_X,cR[0])
measure_2(qc2, n_shots=1000, Binary=True, probability=True)

{'0': 231, '1': 769}


difference in OD bit:  
```
OD bit c0[1]:  
1000 shots: {'0': 248, '1': 752}  

OD bit c0[0]:  
1000 shots: {'1': 474, '0': 526}  
10000 shots: {'1': 5019, '0': 4981}   
120 shots: {'01': 76, '00': 44}  
```




In [ ]:
# qc2=qc.copy()
# qc2.measure_all()
# measure(qc2,n_shots=100)

{'0000010111111101011110011110000010101101011111110001101011111111111111110011100001000010000100111010110100100011101110010111100101011 00': 49, '1000010111111101011110011110000010101101011111110001101011111111111111110011100001000010000100111010110100101011101110010111100101011 00': 51}
Count: 49
index_X:  0
X
0x2b 0x2f 0x77 0xa4
0x75 0x42 0x8 0xe7
0xff 0x5f 0xe3 0xaf
0x15 0x3c 0xaf 0xbf

00101011 00101111 01110111 10100100
01110101 01000010 00001000 11100111
11111111 01011111 11100011 10101111
00010101 00111100 10101111 10111111


Count: 51
index_X:  1
X
0x2b 0x2f 0x77 0xa5
0x75 0x42 0x8 0xe7
0xff 0x5f 0xe3 0xaf
0x15 0x3c 0xaf 0xbf

00101011 00101111 01110111 10100101
01110101 01000010 00001000 11100111
11111111 01011111 11100011 10101111
00010101 00111100 10101111 10111111




# extra

In [ ]:
#for the first row
word="expa"
for i in word:
  print(bin(ord(i))[2:].rjust(8,'0'),end=" ")

In [ ]:
bin(int('e7',16))

'0b11100111'

In [ ]:
print(hex(int('11100111',2)))

'0xe7'

In [ ]:
print(hex(int('11100111',2))[2:].rjust(2,'0'))
print(hex(int('11100110',2))[2:].rjust(2,'0'))
print(hex(int('11100101',2))[2:].rjust(2,'0'))
print(hex(int('11100100',2))[2:].rjust(2,'0'))
print(hex(int('11100011',2))[2:].rjust(2,'0'))
print(hex(int('11100010',2))[2:].rjust(2,'0'))
print(hex(int('11100001',2))[2:].rjust(2,'0'))
print(hex(int('11100000',2))[2:].rjust(2,'0'))

print(hex(int('11101111',2))[2:].rjust(2,'0'))
print(hex(int('11101110',2))[2:].rjust(2,'0'))
print(hex(int('11101101',2))[2:].rjust(2,'0'))
print(hex(int('11101100',2))[2:].rjust(2,'0'))
print(hex(int('11101011',2))[2:].rjust(2,'0'))
print(hex(int('11101010',2))[2:].rjust(2,'0'))
print(hex(int('11101001',2))[2:].rjust(2,'0'))
print(hex(int('11101000',2))[2:].rjust(2,'0'))

e7
e6
e5
e4
e3
e2
e1
e0
ef
ee
ed
ec
eb
ea
e9
e8


In [ ]:
for i in row0.split():
  print(hex(int(i,2))[2:].rjust(2,'0'))

for i in row1.split():
  print(hex(int(i,2))[2:].rjust(2,'0'))

for i in row2.split():
  print(hex(int(i,2))[2:].rjust(2,'0'))

for i in row3.split():
  print(hex(int(i,2))[2:].rjust(2,'0'))

65
78
70
61
75
42
08
e7
ff
5f
e3
af
15
3c
af
bf
